In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import Tensor

In [3]:
import torch.nn.functional as F
def padding_function(data: Tensor) -> Tensor:
    return F.pad(data, (2, 2, 2, 2), mode='constant', value=0)
    

In [4]:
from torchvision.datasets import MNIST
from torchvision import transforms

composed_transforms = transforms.Compose([
    transforms.ToTensor(), # performs scaling for image datasets between range(0-1)
    #padding_function,  # Pad from 28x28 to 32x32
])

train_val_set = MNIST("./data/train", train=True, transform=composed_transforms, download=True)
test_set = MNIST("./data/test", train=False, transform=composed_transforms, download=True)

train_set, val_set = torch.utils.data.random_split(dataset=train_val_set, lengths=[.9, .1])


### Note 
```
If iteration performed over a dataset with transforms, transform operations cen be triggered, if not, will be triggered on train loop and may slow down the process
```

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
X, y = iter(train_set).__next__()
print(X.shape, y)
print(X[X != 0]) # already scaled between 0-1
transforms.ToPILImage()(X)

torch.Size([1, 28, 28]) 2
tensor([0.3843, 0.9176, 0.9176, 0.9176, 0.2510, 0.3451, 0.7608, 0.2980, 0.2980,
        0.8235, 0.8353, 0.8353, 0.9490, 0.9922, 0.9922, 0.9922, 0.9294, 0.1098,
        0.0667, 0.8078, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
        0.9922, 0.9922, 0.9922, 0.9922, 0.1333, 0.2980, 0.9922, 0.9922, 0.9922,
        0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
        0.1333, 0.1216, 0.8510, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
        0.9922, 0.9922, 0.9922, 0.9922, 0.7216, 0.0549, 0.3686, 0.8275, 0.6863,
        0.4824, 0.4824, 0.6588, 0.9922, 0.9922, 0.9922, 0.9922, 0.7412, 0.0275,
        0.0157, 0.0118, 0.4235, 0.9216, 0.9922, 0.9922, 0.9922, 0.7451, 0.2902,
        0.0196, 0.6275, 0.9922, 0.9922, 0.9922, 0.9922, 0.7059, 0.0471, 0.1843,
        0.6000, 0.9922, 0.9922, 0.9922, 0.9922, 0.6902, 0.0667, 0.2627, 0.9059,
        0.9922, 0.9922, 0.9922, 0.9922, 0.8392, 0.2157, 0.0784, 0.1922, 0.1922,
        0.2784

In [6]:
from torch import nn

class LeNet5(nn.Module):
    
    def __init__(self) -> None:
        super().__init__()
        self.feature_extractor = nn.Sequential(
            # Input -> (batch_size,1,32,32)
            # Layer 1
            nn.Conv2d(
                in_channels=1, 
                out_channels=6,
                kernel_size=5,
                padding=2,
                stride=1), # -> (batch_size,6,28,28)
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2), # -> (batch_size,6,14,14)
            
            # Layer 2
            nn.Conv2d(
                in_channels=6,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=0), # -> (batch_size,16,10,10)
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2), # (batch_size,16,5,5)
        )
        self.fully_connected = nn.Sequential(
            nn.Flatten(), # (batch_size,400)
            nn.Linear(in_features=400, out_features=120), # (batch_size,401200)
            nn.Tanh(),
            nn.Linear(in_features=120, out_features=84), # (batch_size,84)
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=10), # (batch_size,10)
        )
    
    def forward(self, x):
        return self.fully_connected(self.feature_extractor(x))
    
    
        

In [7]:
model = LeNet5()
model

LeNet5(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fully_connected): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Tanh()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Tanh()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [8]:
from torchinfo import summary



summary(model, input_size=(1,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
LeNet5                                   [1, 10]                   --
├─Sequential: 1-1                        [1, 16, 5, 5]             --
│    └─Conv2d: 2-1                       [1, 6, 28, 28]            156
│    └─Tanh: 2-2                         [1, 6, 28, 28]            --
│    └─AvgPool2d: 2-3                    [1, 6, 14, 14]            --
│    └─Conv2d: 2-4                       [1, 16, 10, 10]           2,416
│    └─Tanh: 2-5                         [1, 16, 10, 10]           --
│    └─AvgPool2d: 2-6                    [1, 16, 5, 5]             --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Flatten: 2-7                      [1, 400]                  --
│    └─Linear: 2-8                       [1, 120]                  48,120
│    └─Tanh: 2-9                         [1, 120]                  --
│    └─Linear: 2-10                      [1, 84]                   10,164
│  

In [9]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

learning_rate = 0.001
epochs = 10
batch_size=16

optimizer = Adam(params=model.parameters(), lr=learning_rate)
loss_fn =  CrossEntropyLoss()




In [10]:
train_dataloader = DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=2)

val_dataloader = DataLoader(
    val_set, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=2)

test_dataloader = DataLoader(
    test_set, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=2)

train_dataloader, val_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x23a2a920e10>,
 <torch.utils.data.dataloader.DataLoader at 0x23a2a593390>)

In [12]:
from tqdm import tqdm 
import sys

for epoch in tqdm(range(epochs)):
    
    for batch in tqdm(train_dataloader):
        X_train, y_train = batch
        print(X_train.shape)  
        sys.exit()
        # X_train = X_train.to(device)
        # y_train = y_train.to(device)
        
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = loss_fn()        
        
    """
    for (X_val , y_val) in val_dataloader:
        pass
    """  
    

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([16, 1, 28, 28])


  0%|          | 0/10 [00:06<?, ?it/s]


SystemExit: 

d:\workspace\paper-implementations\venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
